# Import Libraries

In [5]:

from tensorflow.keras.applications import VGG16 #use pre-trained neural network
from tensorflow.keras.models import Sequential #it is used to stack layers linearly
from tensorflow.keras.layers import Dense, Flatten, Dropout#imports used layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator #used for data augmentation
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau #regularization
from tensorflow.keras.utils import get_file#downloading cache file
import pandas as pd

# Count the number of files contained in each subdirectories

In [2]:

def count_files_in_subdirs(directory, set_name):
    counts = {}

    for item in os.listdir(directory):
        item_path = os.path.join(directory, item) #facilitate checking whether the project is a file or a subdirectories

        if os.path.isdir(item_path):
            counts[item] = len(os.listdir(item_path))

    df = pd.DataFrame(counts, index=[set_name])
    return df

 # Load Data

In [3]:
import os
train_dir = 'C:/Users/shahi/OneDrive/Documents/6th sem/Minor project/Code/Try 2/tuning/Dataset/train'
test_dir = 'C:/Users/shahi/OneDrive/Documents/6th sem/Minor project/Code/Try 2/tuning/Dataset/test'

#count the files in the subdirectories of the training directory
train_count = count_files_in_subdirs(train_dir, 'train')
print(train_count)

#count the files in the subdirectories of the testing directory
test_count = count_files_in_subdirs(test_dir, 'test')
print(test_count)

       angry  disgust  fear  happy  neutral   sad  surprise
train   3995      436  4097   7215     4965  4830      3171
      angry  disgust  fear  happy  neutral   sad  surprise
test    958      111  1024   1774     1233  1247       831


 # Load Pre-Trained VGG16 Model

In [6]:
#Load the pre-trained VGG16 model without the top layer
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))


In [5]:
#freeze all layers except the last 4 layers
for layer in vgg16_base.layers[:8]:
    layer.trainable = False
for layer in vgg16_base.layers[8:]:
    layer.trainable = True

# Add Custom Layers

In [6]:
#build the model
#8 layers for feature extraction & several dense layers for classification
model = Sequential()

model.add(vgg16_base)

model.add(Flatten())

model.add(Dense(1024, activation='relu'))  # Increased neurons
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))  # Reduced dropout
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(7, activation='softmax'))

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 1, 1, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │           1,799 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,897,927 (60.65 MB)

 Trainable params: 15,342,599 (58.53 MB)

 Non-trainable params: 555,328 (2.12 MB)

# Compile the Model

In [7]:
#compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Preprocess Data

In [ ]:
#The ImageDataGenerator is used to apply various augmentation techniques to the training data,
#enhancing the model's generalization ability
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3]
)
#Only pixel value rescaling is performed to ensure a fair evaluation of the model's performance.
val_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
#automatically infer class labels from the directory structure, loads images, and performs necessary preprocessing
#The generator can provide an efficient and scalable data flow during model training, helping to improve the model's generalization ability.
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.


In [9]:
#automatically infer class labels from the directory structure and evaluates the model's performance at the end of each training epoch
#The generator provides an efficient and scalable data flow for model validation, helping to monitor the model's performance on unseen data and improve its generalization ability.
val_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical'
)

Found 7178 images belonging to 7 classes.


In [10]:
#show the labels of training and validation class
train_class_labels = train_generator.class_indices
print("Training class labels:", train_class_labels)

validation_class_labels = val_generator.class_indices
print("Validation class labels:", validation_class_labels)

Training class labels: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
Validation class labels: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


# Optimization technique

In [11]:
#monitor the validation accuracy
#prevent the model from overfittng
#stops training at the appropriate time while restoring the best model weights
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

In [12]:
#monitor the validation loss and automatically reduces the learning rate when the loss no longer improves
#help the model find a better optimization path and ensure the effectiveness of the training process
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model

In [13]:
#train the model on training and validation dataset
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stopping, reduce_lr]
)

C:\Users\shahi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2234 - loss: 1.8506

C:\Users\shahi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


898/898 ━━━━━━━━━━━━━━━━━━━━ 1762s 2s/step - accuracy: 0.2234 - loss: 1.8506 - val_accuracy: 0.3419 - val_loss: 1.5750 - learning_rate: 1.0000e-04
Epoch 2/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 1872s 2s/step - accuracy: 0.3347 - loss: 1.6473 - val_accuracy: 0.4519 - val_loss: 1.3912 - learning_rate: 1.0000e-04
Epoch 3/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 1765s 2s/step - accuracy: 0.4094 - loss: 1.5096 - val_accuracy: 0.4558 - val_loss: 1.3670 - learning_rate: 1.0000e-04
Epoch 4/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 1851s 2s/step - accuracy: 0.4368 - loss: 1.4513 - val_accuracy: 0.5184 - val_loss: 1.2329 - learning_rate: 1.0000e-04
Epoch 5/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 1029s 1s/step - accuracy: 0.4790 - loss: 1.3697 - val_accuracy: 0.5311 - val_loss: 1.2133 - learning_rate: 1.0000e-04
Epoch 6/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 832s 877ms/step - accuracy: 0.4940 - loss: 1.3438 - val_accuracy: 0.5302 - val_loss: 1.2146 - learning_rate: 1.0000e-04
Epoch 7/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 768s 855ms/step - acc

# Working with test data and evaluating test accuracy

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),  # Replace with your model's input size
    batch_size=32,
    class_mode='categorical'
)


Found 7178 images belonging to 7 classes.


In [15]:
test_loss, test_accuracy = model.evaluate(test_generator, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")



225/225 - 80s - 357ms/step - accuracy: 0.6605 - loss: 0.9555
Test Accuracy: 66.05%


# Save your trained model

In [16]:

model.save('emotion_recognition_model.h5')
print("Model saved successfully!")


Model saved successfully!
